In [1]:
import os
from typing import TypedDict, Annotated, List, Dict, Literal, Union
from langgraph.graph import StateGraph, END, START
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
import asyncio

In [ ]:
# Load API key
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = ""
# os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
# --- Define the state ---
class TravelState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "Conversation messages"]
    budget: float
    destination: str
    itinerary: str
    results: Dict[str, any]

# --- Agent 1: Select best destination ---
async def destination_selector(state: TravelState) -> TravelState:
    print("Please enter your travel budget (in USD):")
    budget = float(input("Your input: "))
    
    # Call LLM to suggest destination
    prompt = f"Suggest the best travel destination under ${budget} for a vacation only one destination."
    response = llm.invoke(prompt)
    
    print(f"Recommended destination: {response.content}")
    
    return {
        **state,
        "messages": state['messages'] + [HumanMessage(content=prompt), AIMessage(content=response.content)],
        "budget": budget,
        "destination": response.content.strip()
    }

# --- Agent 2: Generate itinerary ---
async def itinerary_planner(state: TravelState) -> TravelState:
    print(f"What are your interests for visiting {state['destination']} (comma-separated)?")
    interests = input("Your input: ")
    interests_list = [i.strip() for i in interests.split(',')]
    
    prompt = (
        f"You are a travel assistant. Create a detailed **day-by-day 3-day itinerary** "
        f"for {state['destination']} focusing on: {', '.join(interests_list)}. "
        f"Include morning, afternoon, and evening activities each day, "
        f"with local recommendations and brief descriptions."
    )
    
    response = llm.invoke(prompt)
    
    print("\n🗓️ Generated Itinerary:\n")
    print(response.content)
    
    return {
        **state,
        "messages": state['messages'] + [HumanMessage(content=prompt), AIMessage(content=response.content)],
        "itinerary": response.content.strip()
    }


# --- Coordinator agent ---
async def coordinator_agent(state: TravelState) -> Dict:
    return {
        "results": {
            "required_agents": ["DESTINATION_SELECTOR", "ITINERARY_PLANNER"],
            "priority": {"DESTINATION_SELECTOR": 1, "ITINERARY_PLANNER": 2},
            "concurrent_groups": [["DESTINATION_SELECTOR"], ["ITINERARY_PLANNER"]]
        }
    }

# --- Agent Executor ---
class AgentExecutor:
    def __init__(self):
        self.agents = {
            "DESTINATION_SELECTOR": destination_selector,
            "ITINERARY_PLANNER": itinerary_planner
        }

    async def execute(self, state: TravelState) -> Dict:
        analysis = state["results"].get("coordinator_analysis", {})
        required_agents = analysis.get("required_agents", [])
        results = {}
        
        for agent_name in required_agents:
            agent_func = self.agents.get(agent_name)
            if agent_func:
                state = await agent_func(state)
                results[agent_name.lower()] = "completed"
        
        return {"results": {"agent_outputs": results}}


In [4]:
def create_travel_graph() -> StateGraph:
    graph = StateGraph(TravelState)
    
    graph.add_node("destination_selector", destination_selector)
    graph.add_node("itinerary_planner", itinerary_planner)
    
    graph.set_entry_point("destination_selector")
    graph.add_edge("destination_selector", "itinerary_planner")
    graph.add_edge("itinerary_planner", END)
    
    return graph.compile()

# --- Run the system ---
async def run_travel_planner():
    state = {
        "messages": [HumanMessage(content="I want to plan a trip.")],
        "budget": 0,
        "destination": "",
        "itinerary": "",
        "results":{}
    }
    
    graph = create_travel_graph()
    async for step in graph.astream(state):
        pass  # The agents handle printing themselves


In [ ]:
# To launch
import nest_asyncio
import asyncio

nest_asyncio.apply()
await run_travel_planner()

Please enter your travel budget (in USD):
Recommended destination: One excellent travel destination under $1200 for a vacation is Mexico, specifically Cancun. With its stunning beaches, vibrant nightlife, and rich cultural experiences, Cancun offers a diverse range of activities to suit various interests.

You can typically find round-trip flights from major cities in the U.S. for about $300 to $500. Accommodations range from budget hotels to all-inclusive resorts and can be found for around $60 to $150 per night. Additionally, Cancun's local attractions, such as visiting the Mayan ruins at Tulum, exploring cenotes, or taking a day trip to Isla Mujeres, can be done on a budget.

With careful planning, you can enjoy a wonderful 5- to 7-day vacation in Cancun well within your budget of $1200.
What are your interests for visiting One excellent travel destination under $1200 for a vacation is Mexico, specifically Cancun. With its stunning beaches, vibrant nightlife, and rich cultural exper